# WonkyTops
### Tom Wade, Floris Strijbos, Marina Ten

In [ ]:
import pandas as pd
import numpy as np
import ipyvolume as ipv
import bruges
import os
import math
from scipy.interpolate import griddata
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
cwd = os.getcwd()
print(cwd)

In [ ]:
#Set the location of the input spreadsheet.
xlIn = "sample_data/11_2.xlsx"
xlPath = os.path.join(cwd, xlIn)

In [ ]:
#Read Input tops data from excel file and get the headers
df = pd.read_excel(xlPath)

In [ ]:
#Get a 'list' of all the unique tops found in the input dataset
pd.unique(df.Top)

In [ ]:
#Create List of the Tops to be included:
tops_list = ['AS11.2','T AS11.2']

In [ ]:
def extract_xyz(data, tops):
    """
    Takes the raw dataframe and a list of desired tops, and returns the X, Y and Z
    values as individual arrays (filtering on the tops in the tops list)
    """
    xyz=df.loc[df.Top.isin(tops_list), ['Easting','Northing','Depth_tvdss']].values
    return xyz[:,0] , xyz[:,1] , xyz[:,2]

In [ ]:
x, y, z = extract_xyz(df, tops_list)

In [ ]:
def data_extent(xIn , yIn, zIn):
    """
    Given the X, Y and Z data arrays, this function returns the data extent as a list in the form:
    [0] = xmin
    [1] = xmax
    [2] = ymin
    [3] = ymax
    [4] = zmin
    [z] = zmax
    """
    extent = []
    extent.append(x.min()) ; extent.append(x.max()) ; extent.append(y.min()) ; extent.append(y.max())
    extent.append(z.min()) ; extent.append(z.max())
    return extent

In [ ]:
dExtent = data_extent(x,y,z)

### Initial processing to mathematically remove obvious outliers

In [ ]:
def outliers_iqr(n):
    """
    The interquartile range (IQR), is a measure of statistical dispersion, being equal to the difference 
    between 75th and 25th percentiles, or between upper and lower quartiles, IQR = Q3 − Q1.
    It is a measure of the dispersion similar to standard deviation or variance, but is much more robust 
    against outliers.
    """
    quartile_1, quartile_3 = np.percentile(n, [25, 75])
    iqr = quartile_3 - quartile_1
    lower_bound = quartile_1 - (iqr * 1.5)
    upper_bound = quartile_3 + (iqr * 1.5)
    return np.where((n > upper_bound) | (n < lower_bound))

In [ ]:
def apply_iqr(xIn, yIn, zIn):
    """
    Apply the interquartile range filtering given X,Y and Z arrays of the data
    Returns a 2d numpy array of filtered x, y and z
    """
    iqr_test = outliers_iqr(zIn)[0].tolist()
    x_smooth, y_smooth, z_smooth = np.delete(xIn,iqr_test), np.delete(yIn,iqr_test), np.delete(zIn,iqr_test)
    return np.column_stack((x_smooth, y_smooth, z_smooth))

In [ ]:
#Create filtered data using IQR data set. Pass function X,Y & Z
iqr_data = apply_iqr(x,y,z)

In [ ]:
def iplot(data, extent):
    """
    This Creates the interactive ipython plot, given an array of data.
    This can then be used to interactively edit the dataset. Note it returns an ipv scatter object
    which contains the filter 'selection', which can then be applied
    """
    
    ipv.figure()
    s = ipv.scatter(data[:,0], data[:,1], data[:,2], marker='sphere', size=3)
    ipv.xlim(extent[0], extent[1])
    ipv.ylim(extent[2] ,extent[3])
    ipv.zlim(extent[4],extent[5])
    ipv.animation_control(s)# shows controls for animation controls
    ipv.selector_default()
    ipv.show()
    return s

In [ ]:
#Create the interactive plot, and also return the ipyvol scatter object
scat = iplot(iqr_data, dExtent)

### Iterative interactive editing

In [ ]:
def delete_selected(s):
    """
    This function takes a scatter object returned from the function iplot, and then
    returns a filtered numpy arraty of xyz which can then be reploted
    """
    xFilt = np.delete(s.x, s.selected)
    yFilt = np.delete(s.y, s.selected)
    zFilt = np.delete(s.z, s.selected)
    return np.column_stack((xFilt, yFilt, zFilt))

In [ ]:
#Filter the dataset on the selected points, and reassign to variable
filtered = delete_selected(scat)

In [ ]:
#Redraw the plot with the filtered data
scat = iplot(filtered, dExtent)

In [ ]:
#Refilter the data on any new selection made in the ipyvolume plot
filtered = delete_selected(scat)

In [ ]:
#Replot
iplot(filtered, dExtent)

### Note, you can keep interactively editing the data using the iplot and delete selected functions
An item to be developed is to keep all interactive editing in one plot, without having to keep recalling/redrawinh it.

### Grid Tops via minimum curvature
##### Note this was our first gridding approach. However, a depopulated appoach is going to be preferred...

In [ ]:
def create_grid_outline(raw_extent, spacing):
    """
    Takes the extent of the input data (x,y,z) as a list, and the desired spacing of the output grid
    Returns the X and Y arrays of the grid, in a regularly spaced interval
    """
    #Determine rounded corner points to define the grid
    extent_round = []
    for i in range(len(raw_extent)):
        if i % 2 == 0:
            extent_round.append(int(math.floor(raw_extent[i] / float(spacing))) * spacing)
            continue
        extent_round.append(int(math.ceil(raw_extent[i] / float(spacing))) * spacing)
    nX_points = int((extent_round[1] - extent_round[0]) / spacing)
    nY_points = int((extent_round[3] - extent_round[2]) / spacing)
    grid_x, grid_y = np.mgrid[extent_round[0]:extent_round[1]:complex(nX_points), extent_round[2]:extent_round[3]:complex(nY_points)]
    return grid_x, grid_y, extent_round
    
    

In [ ]:
#Create regularly spaced grid axis
gridX, gridY, grid_extent = create_grid_outline(dExtent, 100)

In [ ]:
def point_coordinates(data):
    """
    Returns the coordinates of an input XYZ array as an array of coordinate pairs in a list
    """
    return np.array(list(zip(data[:,0], data[:,1])))
    

In [ ]:
#Set coordinate points and z values ready to be interpolated
points = point_coordinates(filtered)
values = filtered[:,2]

In [ ]:
def grid_surface(points, values, xgrid, ygrid, intMeth):
    """
    Used Scipy to interpolate points data in to a predefined grid.
    Required a set of coordinate points, a 1d array of depths, a
    x-axis and y-axis defining the grid to interpolate in to.
    Utilised Scipy's 'griddate' interpolator. Different methods can be passed:
    -cubic
    -linear
    -nearest    
    """
    return griddata(points, values, (xgrid, ygrid), method=intMeth)

In [ ]:
#Create an interpolated grid from input data in to regular grid
grid = grid_surface(points, values, gridX, gridY, intMeth='cubic')

In [ ]:
def plot_surface(gridData, extent, topsData, **kwargs):
    """
    Creates a plot of the gridded up well tops, plots the original well tops points
    Takes an optional keyword argument 'res' if residuals are to be plotted.
    This takes residuals as an X,Y,Z(residal) array
    Optional keyword arguments are a contour interval (required to plot contours)
    and original well tops (To compare current grid to original input points)
    """
    plt.figure(figsize=(20,10))
    plt.imshow(gridData.T, origin='lower', extent=extent[:4], cmap='jet')
    plt.colorbar()  
    
    #Plot the cleaned up well tops as a scatter over the top of the grid
    topX, topY = topsData[:,0] , topsData[:,1]
    plt.scatter(topX, topY, s=20, c='k')
    
    #Plot residuals if calculated and in kwargs
    if 'res' in kwargs:
        xRes = kwargs.get("res")[:,0]
        yRes = kwargs.get("res")[:,1]
        residuals = kwargs.get("res")[:,3]
        res_list = residuals.tolist()
        res_txt = [str(round(i, 2)) for i in res_list]

        for i, txt in enumerate(res_txt):
            plt.annotate(txt, (xRes[i], yRes[i]) , size=10)
            
    #Plot Contours
    if 'cont_int' in kwargs:
        contourInt = kwargs.get("cont_int")
        contourMin = round(int(np.nanmin(gridData)), -1)
        contourMax = round(int(np.nanmax(gridData)), -1)
        contours = np.arange(contourMin, contourMax, contourInt)
        plt.contour(gridData.T, extent=extent[:4], levels=contours, colors='k', linestyles='-', linewidths=1)
        
    if 'original' in kwargs:
        originalTops = kwargs.get("original")
        xorig, yorig = originalTops[:,0] , originalTops[:,1]
        plt.scatter(xorig, yorig, s=20, color='gray')
        


### Residuals Extraction

In [ ]:
def calculate_residuals(gExt, gridIn, tops):
    """
    Extract Residuals given a grid, its extent and the well tops
    Return X,Y,Z AND Residual as numpy array
    """
    corners_xy = np.array([[gExt[0], gExt[2]],
                       [gExt[0], gExt[3]],
                       [gExt[1], gExt[2]]])
    
    corners_ix = np.array([[0,  0],
                       [0, gridIn.shape[1]],
                       [gridIn.shape[0],0]])
    
    transform = bruges.transform.CoordTransform(corners_ix, corners_xy)
    
    x = tops[:,0].tolist()
    y = tops[:,1].tolist()
    xyMerge = list(zip(x,y))
    
    coordinatesOut = [transform.reverse([i[0], i[1]]) for i in xyMerge]
    
    xcors = np.array([item[0] for item in coordinatesOut])
    ycors = np.array([item[1] for item in coordinatesOut])
    
    zGrid = gridIn[(xcors, ycors)]
    z = tops[:,2]
    
    residuals = z - zGrid
    
    return np.column_stack((tops[:,0], tops[:,1], tops[:,2], residuals))

In [ ]:
#Get the residuals data. The difference between well top and gridded surface
residuals = calculate_residuals(grid_extent, grid, filtered)

In [ ]:
plot_surface(grid, dExtent, filtered, res=residuals)

### Residual Histogram

In [ ]:
def plot_hist(data):
    """
    Plot a histogram of the input data. It removed nans which the binning doesnt like
    """
    #Dodgy function to account for our data being a 1d series of z values, or an XYZ arrat
    if data.ndim == 2:
        data = data[:,3]
    data = data[~np.isnan(data)]
    rng = np.nanmin(data), np.nanmax(data)
    n, bins, _ = plt.hist(data, bins='auto',  range=rng)

In [ ]:
plot_hist(residuals)

### Regrid using data with histogram outliers kicked out
This part of the workflow needs thought... Understand outliers. Got back to well tops etc
There may be a good case not to run this part of the workflow..!

In [ ]:
def filter_on_condition(data, upper, lower):
    """
    Returns the data with outliers above and below given values removed
    Need to specify the upper and lower bounds when calling the function
    Takes the 2d array (XYZ) and returns an XYZ. This keeps the coordinates so residuls can be regridded/reused
    """
    
    #Remove nans
    data = data[~np.isnan(data[:,3])]
    
    #Apply upper and lower cutoffs
    data = (data[data[:,3] < upper , :])
    data = data[data[:,3] > lower , :]
    
    return data

In [ ]:
residuals_filtered = filter_on_condition(residuals, 5, -5)

In [ ]:
#Rerun histogram to check outliers are removed
plot_hist(residuals_filtered)

In [ ]:
#Set coordinate points and z values ready to be interpolated
points = point_coordinates(residuals_filtered)
values = residuals_filtered[:,2]

In [ ]:
#Recreate the surface using the refiltered tops
grid = grid_surface(points, values, gridX, gridY, intMeth='cubic')

In [ ]:
#Replot the surface
plot_surface(grid, dExtent, residuals_filtered, cont_int=5, original=np.column_stack((x,y,z)))

### Output grid as CSV

In [ ]:
def grid_to_csv(gridx, gridy, grid_data, infPath):
    """
    Function to write the final grid to a CSV file
    """
    outPath = os.path.splitext(infPath)[0] + '_regridded.csv'
    datagrid = pd.DataFrame({'Easting': gridx.ravel(),
                             'Northing': gridy.ravel(),
                             'Values': -grid_data.ravel()},
                            columns=['Easting', 'Northing','Values'])
    datagrid[np.isnan(datagrid)] = 0
    datagrid.to_csv(outPath, index=False)

In [ ]:
grid_to_csv(gridX, gridY, grid, xlPath)

### KDTree

In [ ]:
from scipy import spatial

In [ ]:
#Create sensible grid outline of integers. Now hardcoded...
extent = [596000, 628500, 6640000, 6700000] #Sensible numbers from observed Xmin and Ymin
grid_int = 2000 #Hard coded grid spacing
#Create arrays of the x and y axis
x_points = int((extent[1] - extent[0]) / grid_int)
y_points = int((extent[3] - extent[2]) / grid_int)

In [ ]:
#Create the grid. interval expressed by complex numbers
grid_x, grid_y = np.mgrid[extent[0]:extent[1]:complex(x_points), extent[2]:extent[3]:complex(y_points)]

In [ ]:
#Create numpy arrays of point co-ordinates and then values from input data
points = np.array(list(zip(x, y)))
values = z

In [ ]:
#create kd index
tree = spatial.KDTree(points.tolist())

In [ ]:
keep=list(set(tree.query(list(zip(grid_x.ravel(), grid_y.ravel())))[1]))
xk,yk,zk = x[keep],y[keep],z[keep]

In [ ]:
inds = [i for i in range(x.size)]
blinds = [i for i in inds if i not in keep]
xb, yb, zb = x[blinds] , y[blinds] , z[blinds]

In [ ]:
plt.scatter(xb, yb, color = 'r', s=40, label='blind test')
plt.scatter(xk, yk, color='blue', s=20, label='keep')
plt.legend()

In [ ]:
keepdist=list(set(tree.query(list(zip(grid_x.ravel(), grid_y.ravel())))[0]))

In [ ]:
#Create numpy arrays of point co-ordinates and then values from input data
points = np.array(list(zip(xk, yk)))
values = zk

In [ ]:
#Perform interpolation of data in to the specified grid using scipy
gridded = griddata(points, values, (grid_x, grid_y), method='cubic')

In [ ]:
plt.figure(figsize=(20,10))
plt.imshow(gridded.T, origin='lower', extent=extent, cmap='jet')
plt.colorbar()


plt.scatter(xk, yk, s=10, c='k', label = 'Keep')
plt.scatter(xb, yb, s=10, c = 'r', label='Blind Test')
plt.legend()

### KD Residuals

In [ ]:
gridded.shape

In [ ]:
# The inline, crossline locations you just provided. Also Grid extent, but as indices given by grid.shape
corners_ix = np.array([[0,  0],
                       [0, 29],
                       [15, 0]])

In [ ]:
transform = bruges.transform.CoordTransform(corners_ix, corners_xy)

In [ ]:
#Re-extract residuals
kdXcor, kdYcor = get_top_coords(xk, yk)

In [ ]:
#Extract the value of the grid at the well top locations
kdGridPoint = gridded[(kdXcor, kdYcor)]
kdResiduals = zk - kdGridPoint

In [ ]:
kd_dict = {'x' : xk,
          'y' : yk,
          'z' : zk,
          'zkd' : kdGridPoint,
          'kdResidual' : kdResiduals} 

In [ ]:
kd_df = pd.DataFrame(data=kd_dict)

In [ ]:
#Update histogram
kd_res_list = kdResiduals.tolist()
rng3 = np.nanmin(kd_res_list), np.nanmax(kd_res_list)
n, bins, _ = plt.hist(kd_res_list, bins='auto',  range=rng3)

In [ ]:
bins = (bins[1:] + bins[:-1]) / 2

In [ ]:
plt.bar(bins, n, width=1, color='g')

In [ ]:
#Blind Test Residuals
BkdXcor, BkdYcor = get_top_coords(xb, yb)

In [ ]:
#Extract the value of the grid at the well top locations
BkdGridPoint = gridded[(BkdXcor, BkdYcor)]
BkdResiduals = zb - BkdGridPoint

In [ ]:
#Update histogram
Bkd_res_list = BkdResiduals.tolist()
rng4 = np.nanmin(Bkd_res_list), np.nanmax(Bkd_res_list)
n, bins, _ = plt.hist(Bkd_res_list, bins='auto',  range=rng4)

In [ ]:
bins = (bins[1:] + bins[:-1]) / 2

In [ ]:
plt.bar(bins, n, width=1, color='g')